In [37]:
import pandas as pd 
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import preprocessing
from sklearn.preprocessing import PolynomialFeatures
app_dir = '/Users/nikhilvs/repos/nyu/covid-19'

In [123]:
df = pd.read_csv(os.path.join(app_dir, 'data/staging/unified.csv'))
us_df = df[df['country_region'] == 'Spain']

In [124]:
df.dtypes

country_region                                         object
date                                                   object
retail_and_recreation_percent_change_from_baseline    float64
grocery_and_pharmacy_percent_change_from_baseline     float64
parks_percent_change_from_baseline                    float64
transit_stations_percent_change_from_baseline         float64
workplaces_percent_change_from_baseline               float64
residential_percent_change_from_baseline              float64
C1_School closing                                     float64
C2_Workplace closing                                  float64
C3_Cancel public events                               float64
C4_Restrictions on gatherings                         float64
C5_Close public transport                             float64
C6_Stay at home requirements                          float64
C7_Restrictions on internal movement                  float64
C8_International travel controls                      float64
E1_Incom

In [289]:
TRAIN_COLUMNS = [
    # 'country_region',
    # 'date',
    'retail_and_recreation_percent_change_from_baseline',
    'grocery_and_pharmacy_percent_change_from_baseline',
    'parks_percent_change_from_baseline',
    'transit_stations_percent_change_from_baseline',
    'workplaces_percent_change_from_baseline',
    'residential_percent_change_from_baseline',
    'C1_School closing',
    'C2_Workplace closing',
    'C3_Cancel public events',
    'C4_Restrictions on gatherings',
    'C5_Close public transport',
    'C6_Stay at home requirements',
    'C7_Restrictions on internal movement',
    'C8_International travel controls',
    'E1_Income support',
    'E2_Debt/contract relief',
    'H1_Public information campaigns',
    'H2_Testing policy',
    'H3_Contact tracing',
    'StringencyIndex',
    # 'cases_lag_1',
    # 'cases_lag_2',
    # 'cases_lag_3',
    # 'cases_lag_4',
    # 'cases_lag_5',
    # 'cases_lag_6',
    'cases_lag_7'
    # 'cases_lag_8',
    # 'cases_lag_9',
    # 'cases_lag_10'
]

TARGET_COLUMNS = [
    'cases'
]

df = pd.read_csv(os.path.join(app_dir, 'data/staging/unified.csv'))
us_df = df[df['country_region'] == 'Spain']

COUNTRIES = [
    'United States', 
    'India', 
    'Italy', 
    'Canada', 
    'Spain', 
    'France', 
    'United Kingdom',
    'Iraq',
    'Japan',
    'South Korea',
    'Germany',
    'Brazil'
]

In [247]:
train_subset = us_df[us_df['date'] <= '2020-05-05']
test_subset = us_df[us_df['date'] > '2020-05-05']

# scaler = preprocessing.StandardScaler().fit()
X_train = train_subset[TRAIN_COLUMNS]
y_train = train_subset[TARGET_COLUMNS]
X_test = test_subset[TRAIN_COLUMNS]
y_test = test_subset[TARGET_COLUMNS]

# Naive linear regression - no shift in number of cases
# reg = LinearRegression().fit(X_train, y_train)
reg = Ridge(alpha = 1).fit(X_train, y_train)
print(reg.score(X_train, y_train))
pred = reg.predict(X_test)
print(mean_squared_error(y_test, pred))
print(r2_score(y_test, pred))

0.9941369317818961
12396135.708571726
-6.839137678096982


In [290]:
# for SHIFT_IMPACT_DAYS in list(range(1, 10)):
SHIFT_IMPACT_DAYS = 7
all_countries_df = None

for country in COUNTRIES:
    us_shifted_df = df[df['country_region'] == country].copy()

    SHIFT_COL_TARGET = 'cases_' + str(SHIFT_IMPACT_DAYS)
    us_shifted_df[SHIFT_COL_TARGET] = us_shifted_df['cases'].shift(-SHIFT_IMPACT_DAYS)

    for lag in range(1, 11):
        SHIFT_COL = f'cases_lag_{lag}_' + str(SHIFT_IMPACT_DAYS)
        us_shifted_df[SHIFT_COL] = us_shifted_df[f'cases_lag_{lag}'].shift(-SHIFT_IMPACT_DAYS)

    us_shifted_df = us_shifted_df.dropna()
    if all_countries_df is None:
        all_countries_df = pd.DataFrame(columns = us_shifted_df.columns)
    
    all_countries_df = all_countries_df.append(us_shifted_df)



In [252]:
all_countries_df

,country_region,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,C1_School closing,C2_Workplace closing,...,cases_lag_1_7,cases_lag_2_7,cases_lag_3_7,cases_lag_4_7,cases_lag_5_7,cases_lag_6_7,cases_lag_7_7,cases_lag_8_7,cases_lag_9_7,cases_lag_10_7
850,United States,2020-02-15,6.073507,2.225115,5.158882,1.633614,0.360260,-0.191041,0.0,0.0,...,15.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,12.0
851,United States,2020-02-16,7.348642,2.333067,5.500399,1.421326,0.386581,-0.217652,0.0,0.0,...,15.0,15.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0
852,United States,2020-02-17,3.351574,-0.296495,6.552217,0.668455,-17.336910,2.294349,0.0,0.0,...,15.0,15.0,15.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0
853,United States,2020-02-18,-0.170090,-1.094775,1.335495,0.843243,-0.005405,0.455856,0.0,0.0,...,51.0,15.0,15.0,15.0,13.0,13.0,13.0,13.0,13.0,13.0
854,United States,2020-02-19,1.836030,0.497663,1.990651,0.935994,0.869831,0.102481,0.0,0.0,...,51.0,51.0,15.0,15.0,15.0,13.0,13.0,13.0,13.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,Brazil,2020-04-28,-49.821429,-14.607143,-42.678571,-52.607143,-30.357143,11.892857,3.0,3.0,...,108620.0,101826.0,97100.0,92202.0,87187.0,79685.0,73235.0,67446.0,63100.0,59324.0
74,Brazil,2020-04-29,-49.892857,-13.464286,-40.035714,-52.071429,-30.178571,15.035714,3.0,3.0,...,115455.0,108620.0,101826.0,97100.0,92202.0,87187.0,79685.0,73235.0,67446.0,63100.0
75,Brazil,2020-04-30,-44.964286,1.357143,-35.857143,-47.964286,-29.500000,13.035714,3.0,3.0,...,126611.0,115455.0,108620.0,101826.0,97100.0,92202.0,87187.0,79685.0,73235.0,67446.0
76,Brazil,2020-05-01,-70.000000,-35.107143,-47.428571,-66.500000,-62.785714,22.500000,3.0,3.0,...,135773.0,126611.0,115455.0,108620.0,101826.0,97100.0,92202.0,87187.0,79685.0,73235.0


In [291]:
us_shifted_df = all_countries_df
train_subset = us_shifted_df[us_shifted_df['date'] <= '2020-04-20']
test_subset = us_shifted_df[us_shifted_df['date'] > '2020-04-20']

X_train = train_subset[TRAIN_COLUMNS]
y_train = train_subset[SHIFT_COL_TARGET]
X_test = test_subset[TRAIN_COLUMNS]
y_test = test_subset[SHIFT_COL_TARGET]

scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

reg = LinearRegression().fit(X_train, y_train)
print(reg.score(X_train, y_train))
pred = reg.predict(X_test)
print(mean_squared_error(y_test, pred))
print(r2_score(y_test, pred))

0.9261086445364369
9114673334.888098
0.9134910822801653


In [292]:
reg.coef_

array([-40490.54111325,   2554.89836447, -12132.38021767,  88543.93516332,
       -33479.5313492 , -10845.47173214,  17576.34312071,   6793.97324472,
         9278.77582011, -10154.34699466,  13560.57243996,  27502.00772936,
        18331.47162238,   8521.78410728,  21635.62692816, -12845.69917955,
         8563.21406603,   5201.08576142,    194.13627545, -65791.52903132,
       105282.90539741])

In [293]:
train_subset = us_shifted_df[us_shifted_df['date'] <= '2020-04-25']
test_subset = us_shifted_df[us_shifted_df['date'] > '2020-04-25']

X_train = train_subset[TRAIN_COLUMNS]
y_train = train_subset[SHIFT_COL_TARGET]
X_test = test_subset[TRAIN_COLUMNS]
y_test = test_subset[SHIFT_COL_TARGET]

poly = PolynomialFeatures(3)

X_train_poly = poly.fit_transform(X_train)
scaler = preprocessing.StandardScaler().fit(X_train_poly)

X_train_poly = scaler.transform(X_train_poly)
X_test_poly = scaler.transform(poly.fit_transform(X_test))

# reg = LinearRegression().fit(X_train_poly, y_train)
reg = Ridge(alpha = 10).fit(X_train_poly, y_train)
print(reg.score(X_train_poly, y_train))
pred = reg.predict(X_test_poly)
print(mean_squared_error(y_test, pred))
print(r2_score(y_test, pred))

0.9987491611776762
728364496.0690948
0.9937625675445343


In [294]:
y_test

921    1158040.0
922    1180375.0
923    1204351.0
924    1229331.0
925    1257023.0
         ...    
73      115455.0
74      126611.0
75      135773.0
76      146894.0
77      156061.0
Name: cases_7, Length: 77, dtype: float64

In [295]:
list(pred)

[1106032.1070010227,
 1123928.3145641827,
 1192156.6899890723,
 1176930.0210366792,
 1189612.7830159143,
 1250585.5207892319,
 1338759.9080732665,
 36824.531482654136,
 37149.41780008577,
 46251.36220915321,
 47065.42837406152,
 48096.311542702635,
 48428.48905767579,
 51396.580209768545,
 211362.91297518087,
 221345.12542223465,
 223276.57989380835,
 226284.32922910978,
 227160.91211140144,
 208350.92063849006,
 233694.39249667237,
 3041.519207879006,
 10159.908588534738,
 24424.10649666265,
 19255.765961842284,
 17995.61985267148,
 15047.079899205331,
 33492.550623664996,
 215720.87216583046,
 220085.26265456443,
 223596.9822428415,
 220857.24293048534,
 223151.84736283222,
 218083.36328734187,
 243769.69831578343,
 35575.71906261453,
 40789.87186846683,
 42092.487973292664,
 41761.43317197379,
 42423.16718727601,
 59079.64566664244,
 6101.73669924905,
 7434.701699798017,
 4783.64669766483,
 7228.868808828243,
 4915.0718498113565,
 3710.9453929451265,
 1751.6666388729864,
 4208.41419